<a href="https://colab.research.google.com/github/Phionanamugga/Autonomous_Navigation/blob/main/ObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_velodyne.zip
!wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_calib.zip

--2025-08-09 13:32:49--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.137.122, 52.219.170.93, 3.5.139.177, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.137.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12569945557 (12G) [application/zip]
Saving to: ‘data_object_image_2.zip’

data_object_image_2 100%[===================>]  11.71G  34.6MB/s    in 6m 15s  

2025-08-09 13:39:05 (32.0 MB/s) - ‘data_object_image_2.zip’ saved [12569945557/12569945557]

--2025-08-09 13:39:05--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.135.110, 52.219.140.111, 3.5.138.189, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.135.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

In [2]:
!unzip -q '*.zip'


4 archives were successfully processed.


In [4]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import pandas as pd
import torch

In [ ]:
!pip install mmcv mmdet mmdet3d --ignore-installed
from mmdet3d.datasets import build_dataset
cfg.data.train.data_root = './data/kitti/'
dataset = build_dataset(cfg.data.train)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:

model = torch.hub.load('pytorch/vision:v0.15.2', 'deeplabv3_resnet101', pretrained=True)
model.eval()


In [ ]:
import torchvision.transforms as T
from PIL import Image
import numpy as np

transform = T.Compose([T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406],
                                               std=[0.229, 0.224, 0.225])])

def segment_image(image_path):
    img = Image.open(image_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    sem_prob = torch.softmax(output, dim=0).cpu().numpy()  # shape: [21, H, W]
    return sem_prob


In [ ]:
import numpy as np

def project_lidar_to_image(points, calib):
    # points: Nx3 array (x,y,z)
    # calib: calibration matrix from KITTI
    # Return: Nx2 array of pixel coordinates
    pts_3d = np.hstack((points, np.ones((points.shape[0],1))))
    pts_2d = calib @ pts_3d.T
    pts_2d[:2, :] /= pts_2d[2, :]
    return pts_2d[:2, :].T  # Nx2 pixel coordinates